In [3]:
from konlpy.tag import Okt
okt=Okt()

file = open('./ratings_test.txt', 'r', encoding='utf-8-sig')

sentences = []
label = []

for idx, line in enumerate(file):
    if idx == 0:
        continue
    line = line.split('\t')
    sentences.append(line[1])
    label.append(line[2].strip())

In [4]:
sentences_pos = []

for line in sentences:
    sentences_pos.append(okt.morphs(line))

In [5]:
print(sentences_pos[:5])

[['굳', 'ㅋ'], ['GDNTOPCLASSINTHECLUB'], ['뭐', '야', '이', '평점', '들', '은', '....', '나쁘진', '않지만', '10', '점', '짜', '리', '는', '더', '더욱', '아니잖아'], ['지루하지는', '않은데', '완전', '막장', '임', '...', '돈', '주고', '보기', '에는', '....'], ['3', 'D', '만', '아니었어도', '별', '다섯', '개', '줬을텐데', '..', '왜', '3', 'D', '로', '나와서', '제', '심기', '를', '불편하게', '하죠', '??']]


In [6]:
max_len = max([len(i) for i in sentences_pos])

vocab = set()

for line in sentences_pos:
    for word in line:
        vocab.add(word)

In [7]:
vocab_size = len(vocab)+1

vocab = sorted(list(vocab))

vocab_index = {}
for i in range(len(vocab)):
    vocab_index[vocab[i]] = len(vocab_index) +1

In [8]:
print(vocab[9990:10000])
print(vocab_index['나서써라'])

['나서써라', '나서야', '나서의', '나선', '나설', '나섰다가', '나성', '나셨어', '나소', '나수윤']
9991


In [9]:
int_sentences = []

for line in sentences_pos:
    etc = []
    for word in line:
        etc.append(vocab_index[word])
    int_sentences.append(etc)

In [10]:
print(int_sentences[:5])

[[7000, 3312], [1790], [21891, 34951, 40013, 52555, 16227, 39776, 406, 9955, 34363, 626, 44517, 47854, 18306, 12735, 14272, 14369, 33161], [47084, 34326, 38030, 18752, 41581, 378, 14894, 45887, 24632, 36379, 406], [1010, 1718, 18782, 33122, 24384, 13187, 4948, 46673, 346, 38146, 1010, 1718, 18049, 10265, 44855, 31885, 18300, 26512, 53798, 1567]]


In [13]:
!pip install keras
!pip install tensorflow
from keras.preprocessing.sequence import pad_sequences

padding_sentences = []
int_sentences = pad_sequences(int_sentences, padding='post', maxlen=max_len)

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Running setup.py install for gast: started
    Running setup.py install for gast: finished with status 'done'
  Running setup.py install for opt-einsum: started
    Running setup.py install for opt-einsum: finished with status 'done'
  Running setup.py install for absl-py: started
    Running setup.py install for absl-py: finished with status 'done'
  Running setup.py install for termcolor: started
    Running setup.py install for termcolor: finished with status 'done'
  Found existing installation: setuptools 40.6.2
    Uninstalling setuptools-40.6.2:
      Successfully uninstalled setuptools-40.6.2
  Running setup.py install for wrapt: started
    Running setup.py install for wrapt: finished with status 'done'


You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Using TensorFlow backend.


In [15]:
print(sentences_pos[5])
print(int_sentences[5])

['음악', '이', '주가', '된', ',', '최고', '의', '음악', '영화']
[39881 40013 45872 15654   203 49613 39934 39881 37170     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0]


In [16]:
from keras.utils.np_utils import to_categorical

label_one_hot = to_categorical(label)

In [17]:
print(label_one_hot[:5])

[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_etc, y_train, y_etc = train_test_split(int_sentences, label_one_hot, test_size = 0.4, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_etc, y_etc, test_size=0.5, random_state=42)

In [19]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

30000
10000
10000


In [20]:
from keras.layers import Embedding, Input, Dense, Flatten #keras에서도 임베딩 레이어를 제공함
from keras.models import Sequential
from keras.models import Model

embedding_size = 64

input_data = Input(shape=(max_len, ))

emd = Embedding(vocab_size, 64)(input_data)

flatten = Flatten()(emd)

dense1 = Dense(64, activation='relu')(flatten)
dense2 = Dense(64, activation='relu')(dense1)
dense3 = Dense(2, activation='softmax')(dense2)

model = Model(inputs=input_data, outputs=dense3)
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 82)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 82, 64)            3635968   
_________________________________________________________________
flatten_1 (Flatten)          (None, 5248)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                335936    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 3,976,194
Trainable params: 3,976,194
Non-trainable params: 0
_________________________________________________

In [21]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose=1)

model.fit([X_train], y_train, batch_size=256, epochs=8, validation_data=([X_val], y_val), callbacks=[es])


c:\python\venv\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 30000 samples, validate on 10000 samples
Epoch 1/8
30000/30000 [==============================] - 12s 406us/step - loss: 0.5754 - accuracy: 0.6706 - val_loss: 0.4729 - val_accuracy: 0.7715
Epoch 2/8
30000/30000 [==============================] - 12s 396us/step - loss: 0.2745 - accuracy: 0.8881 - val_loss: 0.4140 - val_accuracy: 0.8173
Epoch 3/8
30000/30000 [==============================] - 12s 396us/step - loss: 0.1013 - accuracy: 0.9669 - val_loss: 0.5488 - val_accuracy: 0.8030
Epoch 00003: early stopping


In [22]:
evaluation = model.evaluate([X_test], y_test)

print('Loss: '+str(evaluation[0]))
print('Accuracy: '+str(evaluation[1]))

10000/10000 [==============================] - 0s 48us/step
Loss: 0.5649034008979797
Accuracy: 0.8004999756813049
